In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from bs4 import BeautifulSoup
import requests
# from tqdm import tqdm, tqdm_pandas

from tqdm.notebook import tqdm
tqdm.pandas()


In [ ]:
# ! pip install tqdm --upgrade

     |████████████████████████████████| 71kB 2.1MB/s 
  Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1


## 0. data load

In [ ]:
def data_load():
    file_path = "/content/drive/My Drive/Colab Notebooks/2020bigcontest/data/"
    train_data = pd.read_csv(file_path+"train_data_pre1_0820.csv")
    train_data['key_index'] = list(range(len(train_data)))

    test_data = pd.read_csv(file_path+"test_data_pre1_0820.csv")
    test_data['key_index'] = list(range(len(test_data)))

    # train_data.columns = ['time','exp_min','m_code','p_code','p_name','p_cate','p_price','target']
    # test_data.columns = ['time','exp_min','m_code','p_code','p_name','p_cate','p_price','target']

    return file_path, train_data, test_data

## 1. crawling

In [ ]:
def crawl(keyword):
    url = f"https://search.shopping.naver.com/search/all.nhn?query={keyword}&cat_id=&frm=NVSHATC"
    data = requests.get(url)
    return data.content

In [ ]:
def getCrawlInfo(li):
    # get Productname, link, price 
    aTit = li.find("a", {"class": "basicList_link__1MaTN"})
    name = aTit.text
    link = aTit['href']
    price = li.find("span", {"class":"price_num__2WUXn"}).text.replace(",", "")

    # get Categories - 대/ 중/ 소  (없으면 None 으로 표기되도록)
    cate_class = li.findAll("a", {"class": "basicList_category__wVevj"})
    cate_list = [None, None, None]
    for idx in range(3):
        if cate_class[idx] :
            cate_list[idx] = cate_class[idx].text
        else:
            pass

    result_list = [name, link, price] + cate_list 
    return result_list

In [ ]:
def parse(pageString):
    bsObj = BeautifulSoup(pageString, "html.parser")
    ul = bsObj.find("ul", {"class":"list_basis"})

    if ul: # 검색 결과 있는 경우 
        lis = ul.find("li",{"class":"basicList_item__2XT81"}) 
        crawl_info_list = getCrawlInfo(lis)
        crawl_info_list = [1] + crawl_info_list   # 검색 결과 여부도 labeling - (결과 있으면 1)
        # print(crawl_info_list)

    else: # 검색 결과 없는 경우 
        crawl_info_list = [None]*6
        crawl_info_list = [0] + crawl_info_list   # 검색 결과 여부도 labeling - (결과 없으면 0)
        # print(crawl_info_list)

    return pd.Series(crawl_info_list)

In [ ]:
def get_crawl_data(p_name):
    try : 
        pageString = crawl(p_name)
        crawl_info_series = parse(pageString)
    except: 
        crawl_info_series = pd.Series([-1]+['None']*5) # 에러 결과 여부도 labeling -1
    return crawl_info_series

In [ ]:
def make_crawl_df(train_data, test_data):
    # train_data - make DF only for index and text 
    print("train data crawling")
    train_cr_data = train_data[['key_index','p_name','p_cate']]
    train_cr_data[['cr','cr_name','cr_link','cr_price','cr_cate1','cr_cate_2','cr_cate3']] = train_cr_data['p_name'].progress_apply(get_crawl_data)
    print("train data crawling finished")

    # test_data - make DF only for index and text
    print("test data crawling")
    test_cr_data = test_data[['key_index','p_name','p_cate']]
    test_cr_data[['cr','cr_name','cr_link','cr_price','cr_cate1','cr_cate_2','cr_cate']] = test_cr_data['p_name'].progress_apply(get_crawl_data)
    print("test data crawling finished")

    return train_cr_data, test_cr_data

In [ ]:
def main(train_data, test_data):
    # # check sample result 
    # train_data = train_data.iloc[:100,:]
    # test_data = test_data.iloc[:1,:]

    # CRAWLING ! 
    train_cr_data, test_cr_data = make_crawl_df(train_data,test_data)

    return train_cr_data, test_cr_data

In [ ]:
if __name__ == "__main__":
    # 0. data load (preprocessed ver)
    # tqdm.pandas(tqdm())
    file_path, train_data, test_data = data_load()

    # 1. crawling
    train_cr_data, test_cr_data = main(train_data, test_data)

train data crawling



train data crawling finished
test data crawling


/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]



test data crawling finished


In [ ]:
# save file

file_path = "/content/drive/My Drive/Colab Notebooks/2020bigcontest/data/"
train_cr_data.to_csv(file_path+"train_cr_data(ver1_0821).csv", index = False)
test_cr_data.to_csv(file_path+"test_cr_data(ver1_0821).csv", index = False)

## 2. 저장된 파일 확인

In [ ]:
file_path = "/content/drive/My Drive/Colab Notebooks/2020bigcontest/data/"
ck_train_cr_data = pd.read_csv(file_path+"train_cr_data(ver1_0821).csv")
ck_test_cr_data = pd.read_csv(file_path+"test_cr_data(ver1_0821).csv")

In [ ]:
ck_train_cr_data[ck_train_cr_data['cr'] == 1] # 크롤링이 제대로 된 경우  38303중 10436개...

,key_index,p_name,p_cate,cr,cr_name,cr_link,cr_price,cr_cate1,cr_cate_2,cr_cate3
0,0,테이트 남성 셀린니트3종,의류,1.0,TATE 남성 캐시미어 블랜딩 니트3종,https://cr.shopping.naver.com/adcr.nhn?x=%2BAs...,68510원,패션의류,남성의류,니트/스웨터
1,1,테이트 여성 셀린니트3종,의류,1.0,테이트 여성 19FW 헤어리 폭스블렌디드 니트 3종,https://cr.shopping.naver.com/adcr.nhn?x=aqw7X...,39110원,패션의류,여성의류,니트/스웨터
2,2,테이트 남성 셀린니트3종,의류,1.0,TATE 남성 캐시미어 블랜딩 니트3종,https://cr.shopping.naver.com/adcr.nhn?x=OMPHE...,68510원,패션의류,남성의류,니트/스웨터
3,3,테이트 여성 셀린니트3종,의류,1.0,테이트 여성 19FW 헤어리 폭스블렌디드 니트 3종,https://cr.shopping.naver.com/adcr.nhn?x=z0BEL...,39110원,패션의류,여성의류,니트/스웨터
4,4,테이트 남성 셀린니트3종,의류,1.0,TATE 남성 캐시미어 블랜딩 니트3종,https://cr.shopping.naver.com/adcr.nhn?x=fc7pH...,68510원,패션의류,남성의류,니트/스웨터
...,...,...,...,...,...,...,...,...,...,...
38299,38299,일시불쿠첸압력밥솥 6인용,주방,1.0,쿠첸 6인용 열판압력밥솥 CJS-FA0602V,https://cr.shopping.naver.com/adcr.nhn?x=qD%2B...,111240원,디지털/가전,주방가전,전기밥솥
38300,38300,무이자쿠첸압력밥솥 10인용,주방,1.0,쿠첸 10인용 1등급 전기압력밥솥 CJS-FC1003F,https://cr.shopping.naver.com/adcr.nhn?x=8f%2F...,110110원,디지털/가전,주방가전,전기밥솥
38301,38301,일시불쿠첸압력밥솥 10인용,주방,1.0,쿠첸 10인용 1등급 전기압력밥솥 CJS-FC1003F,https://cr.shopping.naver.com/adcr.nhn?x=9X1Mo...,110110원,디지털/가전,주방가전,전기밥솥
38302,38302,무이자쿠첸압력밥솥 6인용,주방,1.0,쿠첸 6인용 열판압력밥솥 CJS-FA0602V,https://cr.shopping.naver.com/adcr.nhn?x=t9uZZ...,111240원,디지털/가전,주방가전,전기밥솥


In [ ]:
def data_load():
    file_path = "/content/drive/My Drive/Colab Notebooks/2020bigcontest/data/"
    train_cr_data = pd.read_csv(file_path+"train_cr_data(ver1_0821).csv")

    test_cr_data = pd.read_csv(file_path+"test_cr_data(ver1_0821).csv")

    # train_data.columns = ['time','exp_min','m_code','p_code','p_name','p_cate','p_price','target']
    # test_data.columns = ['time','exp_min','m_code','p_code','p_name','p_cate','p_price','target']

    return file_path, train_cr_data, test_cr_data

In [ ]:
file_path, train_data, test_data = data_load()

In [ ]:
train_data[train_data['cr']==0].head(50)

,key_index,p_name,p_cate,cr,cr_name,cr_link,cr_price,cr_cate1,cr_cate_2,cr_cate3
6,6,오모떼 레이스 파운데이션 브라,속옷,0.0,NaN,NaN,NaN,NaN,NaN,NaN
7,7,오모떼 레이스 파운데이션 브라,속옷,0.0,NaN,NaN,NaN,NaN,NaN,NaN
8,8,오모떼 레이스 파운데이션 브라,속옷,0.0,NaN,NaN,NaN,NaN,NaN,NaN
12,12,보코 리버시블 무스탕,의류,0.0,NaN,NaN,NaN,NaN,NaN,NaN
13,13,보코 리버시블 무스탕,의류,0.0,NaN,NaN,NaN,NaN,NaN,NaN
14,14,보코 리버시블 무스탕,의류,0.0,NaN,NaN,NaN,NaN,NaN,NaN
25,25,그렉노먼 남성 구스다운 롱 벤치코트,의류,0.0,NaN,NaN,NaN,NaN,NaN,NaN
26,26,그렉노먼 여성 구스다운 롱 벤치코트,의류,0.0,NaN,NaN,NaN,NaN,NaN,NaN
27,27,그렉노먼 남성 구스다운 롱 벤치코트,의류,0.0,NaN,NaN,NaN,NaN,NaN,NaN
28,28,그렉노먼 여성 구스다운 롱 벤치코트,의류,0.0,NaN,NaN,NaN,NaN,NaN,NaN
